In [ ]:
import sys, random
import numpy as np
import matplotlib.pyplot as plt

9


In [ ]:
### Parámetros

n_rondas = 100

# Pagos
socios = 3
aprovechado = 5
traidores = 1
tonto = 0

# Extras
ruido = 0.0 # probabilidad de voltear acción elegida. Ingresar valores entre [0, 1]
semilla = 42 # para funciones con aleatoriedad

# Acciones
C = "C" # cooperar
T = "T" # traicionar
acciones = (C, T)

In [ ]:
# Matriz / Función de PAGOS

PAGOS = {
        (C, C): (socios, socios),
        (C, T): (tonto, aprovechado),
        (T, C): (aprovechado, tonto),
        (T, T): (traidores, traidores)}

def pago_ronda(a_i, a_j, pagos = PAGOS):
    #Devuelve (pago_i, pago_j) según las acciones de la ronda

    # Por ejemplo:
    # pago_ronda("C", "C") = (3, 3)
    # pago_ronda("C", "T") = (0, 5)

    return pagos[(a_i, a_j)]

In [ ]:
class Estrategia:
    nombre = "base"
    def reiniciar(self): # Reinicia el "tablero" del juego
        return None

    def decidir(self, mi_historial, historial_rival):
        raise NotImplementedError("Implementa decidir() en la subclase.")

# 10 estrategias, 10 jugadores:
class OjoPorOjo(Estrategia):
    nombre = "Ojo por ojo"
    def decidir(self, mi_historial, historial_rival):
        # Primera ronda coopera, luego copia la última decisión del rival
        if len(historial_rival) == 0:
            return C
        else:
            return historial_rival[-1]

class SiempreCooperar(Estrategia):
    nombre = "Siempre cooperar"
    def decidir(self, mi_historial, historial_rival):
        return C

class SiempreTraicionar(Estrategia):
    nombre = "Siempre traicionar"
    def decidir(self, mi_historial, historial_rival):
        return T
    
class Aleatorio(Estrategia):
    nombre = "Aleatorio"
    def decidir(self, mi_historial, historial_rival):
        r = random.random()
        if r <= 0.50:
            return C
        else:
            return T
        
class Vengativo(Estrategia):
    nombre = "Vengativo"
    def decidir(self, mi_historial, historial_rival):
        if len(historial_rival) == 0:
            return C
        
        conteo_traiciones = 0
        for i in range(n_rondas):
            if historial_rival[i] == T:
                conteo_traiciones = conteo_traiciones + 1
        if conteo_traiciones > 0:
            return T
        else:
            return C

In [1]:
# Jugadas con opción de ruido de comunicación

def aplicar_ruido(accion, prob, rng):
    # Con una probabilidad elegida (5.0%, por ejemplo), el jugador escoge la acción contraria a la que normalmente su estrategia le dicta.
    # Objetivo: simular mejor el mundo real en que, por problemas de comunicación, una acción se puede malinterpretar. Analizar cómo se
    # comportan estrategias en ambientes con imperfección.
    
    prob = float(prob)
    if prob == 0.0:
        return accion

    # Voltea decisión con probabilidad p
    return (C if accion == T else T) if rng.random() < prob else accion